In [78]:
import open3d as o3d
import numpy as np
import sys
import math

import pandas as pd
from scipy.spatial.transform import Rotation as R


In [79]:
def get_minmax(pcd_path):
    ##// pcd 데이터 좌표범위 검증용
    pcd_load = o3d.io.read_point_cloud(pcd_path)
    pc_array = np.asarray(pcd_load.points)
    
    x_min = sys.float_info.max
    y_min = sys.float_info.max
    z_min = sys.float_info.max
    x_max = sys.float_info.min
    y_max = sys.float_info.min
    z_max = sys.float_info.min
    
    depth_list = []
    for pt in pc_array:
        x = pt[0]
        y = pt[1]
        z = pt[2]
        
        if x < x_min: x_min = x
        if y < y_min: y_min = y
        if z < z_min: z_min = z
        
        if x > x_max: x_max = x
        if y > y_max: y_max = y
        if z > z_max: z_max = z
        
        dist = math.sqrt( (x**2) + (y**2) + (z**2) )
        depth_list.append(dist)
    
    print("\n x min max")
    print(x_min, x_max)
    
    print("\n y min max")
    print(y_min, y_max)
    
    print("\n z min max") 
    print(z_min, z_max)
    return depth_list, x_min, x_max, y_min, y_max, z_min, z_max



def flip_xy(arr):
    
    arr = pd.DataFrame(arr, columns=['x', 'y', 'z'])

    #arr = arr[['y', 'x', 'z']]
    
    arr['x'] = arr['x'] * (-1)
    arr['y'] = arr['y'] * (-1)
    #arr['z'] = arr['z'] * (-1)
    
    arr = arr.to_numpy()

    return arr


def flip_xyz(pcd_arr, to_flip=None):
    ##// input np.array shape (N, 3)
    ##// to_flip (str): 'xyz', 'xy'......
    pcd_arr = pd.DataFrame(pcd_arr, columns=['x', 'y', 'z'])
    
    pcd_arr['x'] = pcd_arr['x'] * (-1)
    pcd_arr['y'] = pcd_arr['y'] * (-1)
    
    pcd_arr = pcd_arr.to_numpy()
    
    for i in range(len(pcd_arr)):
        if 'x' in to_flip:
            pcd_arr[i,0] = -(pcd_arr[i,0])    
        if 'y' in to_flip:
            pcd_arr[i,1] = -(pcd_arr[i,1])
        if 'z' in to_flip:
            pcd_arr[i,2] = -(pcd_arr[i,2])

    return pcd_arr



def rotate_pcd(pc_arr, dir_vec, theta):
    ret = []
    
    for pt in pc_arr:
        v = pt * math.cos(theta) + math.sin(theta)*np.cross(dir_vec, pt) + (1-math.cos(theta))*(np.dot(dir_vec, pt))*dir_vec
        ret.append(v)
    ret = np.asarray(ret)
    return ret


def rotate_pcd2(pc_arr, rmat):
    ret = []
    for pt in pc_arr:
        v = rmat @ pt
        ret.append(v)
    ret = np.asarray(ret)
    return ret


def cut_pcd(pc_arr, direction=-1):
    pc_arr = pd.DataFrame(pc_arr, columns=['x', 'y', 'z'])
    #pc_arr = pc_arr[pc_arr['z'] >= -1 ]
    
    if direction==1:
        pc_arr = pc_arr[pc_arr['y'] <= 0]
    elif direction==2:
        pc_arr = pc_arr[pc_arr['x'] >= 0]
    elif direction==3:
        pc_arr = pc_arr[pc_arr['y'] >= 0]
    elif direction==4:
        pc_arr = pc_arr[pc_arr['y'] >= 0]
    elif direction==5:
        pc_arr = pc_arr[pc_arr['x'] <= 0]
    
    pc_arr = pc_arr.to_numpy()
    return pc_arr




In [80]:
lidar_path = "./dataset_sample/lidar/000599.pcd"
radar_path = "./dataset_sample/radar/000599.pcd"

In [81]:
lr = 3.666930
ud = 0.000000
rot_deg = 181.455733
rot_rad = rot_deg * (math.pi / 180)

dir_vec = np.asarray([[-lr, -1.0, ud]]) / math.sqrt(lr**2 + 1 + ud**2)

axis_angle = (dir_vec * rot_deg).reshape((3,1))

r = R.from_euler('xyz', [lr, ud, rot_deg], degrees=True)
rmat = r.as_matrix()
print(rmat)
tvec = np.asarray([0.006550,-2.671000,-2.365720])

[[-9.99677251e-01  2.53525896e-02 -1.62478476e-03]
 [-2.54046004e-02 -9.97630611e-01  6.39356784e-02]
 [-1.08420217e-19  6.39563202e-02  9.97952699e-01]]


In [82]:
pcd_load_lidar = o3d.io.read_point_cloud(lidar_path)
pc_array_lidar = np.asarray(pcd_load_lidar.points)

pcd_load_radar = o3d.io.read_point_cloud(radar_path)
pc_array_radar = np.asarray(pcd_load_radar.points)

#pc_array_radar = rotate_pcd2(pc_array_radar, rmat)
pc_array_radar = flip_xyz(pc_array_radar, 'y')

pc_array_radar = pc_array_radar + tvec

In [83]:
print(pc_array_lidar.shape, pc_array_radar.shape)
pc_array = np.concatenate((pc_array_lidar, pc_array_radar), axis=0)

print(pc_array_lidar.shape, pc_array_radar.shape, pc_array.shape)
lidar_color = np.zeros((len(pc_array_lidar), 3))
radar_color = np.zeros((len(pc_array_radar), 3)) 

(64000, 3) (485, 3)
(64000, 3) (485, 3) (64485, 3)


In [84]:
lidar_color = pd.DataFrame(lidar_color, columns=['r', 'g', 'b'], dtype=np.int64)
lidar_color['b'] = lidar_color['b'] + 255

radar_color = pd.DataFrame(radar_color, columns=['r', 'g', 'b'], dtype=np.int64)
radar_color['r'] = radar_color['r'] + 255

lidar_color = lidar_color.to_numpy()
radar_color = radar_color.to_numpy()

color_array = np.concatenate((lidar_color, radar_color), axis=0)

color_array.shape

(64485, 3)

In [85]:
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(pc_array) # array_of_points.shape = (N,3)
point_cloud.colors = o3d.utility.Vector3dVector(color_array) # array_of_colors.shape = (N,3)

o3d.visualization.draw_geometries([point_cloud])